# ReactiveX in Python - baseline
A baseline experiment with no `async` tomfoolery.
* https://github.com/ReactiveX/RxPY
* https://rxpy.readthedocs.io/en/latest/get_started.html

# *Status :-)*

# Try out with widgets

In [1]:
import ipywidgets as widgets
from IPython.display import display

## Make a widget observable

In [2]:
def make_push_widget(w):
    def push_widget(observer, scheduler):
        # make the callback to attach to the widget
        def on_value_change(change):
            new_value = change['new']
            #print(f"ovc new_value is {new_value}")
            observer.on_next(change['new'])
            if new_value == 100:
                observer.on_completed()
                w.unobserve(on_value_change, names='value')
        # attach callback to widget
        w.observe(on_value_change, names='value')
    # return the observation function
    return push_widget

## Place some widgets 

In [3]:
from math import pi

v_w = widgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description='Sin:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='.2f',
)

h_w = widgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description='Cos:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

t_w = widgets.IntSlider(
    description='t',
)

shutdown_b_w = widgets.Button(description="Shutdown worker")
clear_b_w = widgets.Button(description="clear")
out_w = widgets.Output(layout={'border': '1px solid black'})
display(v_w, h_w, t_w, shutdown_b_w, clear_b_w, out_w)

FloatSlider(value=0.0, continuous_update=False, description='Sin:', max=1.0, min=-1.0, orientation='vertical',…

FloatSlider(value=0.0, continuous_update=False, description='Cos:', max=1.0, min=-1.0, step=0.01)

IntSlider(value=0, description='t')

Button(description='Shutdown worker', style=ButtonStyle())

Button(description='clear', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

## Interaction functions

In [4]:
from math import sin, cos, pi
import rx

def slider_bender(s, v):
    s.value = v

def circulate(t):
    slider_bender(v_w, sin(t))
    slider_bender(h_w, cos(t))

def shutdown_child(w):
    with out_w:
        print(f"would send shutdown from shutdown_child")

def clear_out(w):
    out_w.clear_output()

### Plug in button callbacks

In [5]:
shutdown_b_w.on_click(shutdown_child)
clear_b_w.on_click(clear_out)

## Build ReactiveX pipeline

In [6]:
from rx import operators as op
from math import pi

t_src = rx.create(make_push_widget(t_w))

t_src.pipe(
    op.map(lambda i: i/100),
    op.map(lambda t: 2*pi * 2*t)
).subscribe(
    on_next = circulate,
    on_error = lambda e: print("Error Occurred: {0}".format(e)),
    on_completed = lambda: print("Done!"),
)

### Test
Go up to Widgets and manipulate the **t** slider. When you're done, make the pipeline shut down by sliding it all the way to 100.

## UI while working
Can these controls and responses be used while doing other work, perhaps work that is controlled by the widgets?
Go rerun the pipeline above, verify it works by manipulating the `t` slider, and leave it working. Run the cell below, and again manipulate the slider.

In [ ]:
import asyncio
async def sleep_print():
    for i in range(30):
        print(f"radius is {v_w.value**2 + h_w.value**2}, run {i}")
        await asyncio.sleep(1)

asyncio.create_task(sleep_print())

# Conclusions
* The path from widget -> rx pipeline -> widget works, as long as another code block is not running.
* When the running code block returns, the path works again